# Project Capestone: week 3 question 1

### 1. Configure Environment

In [25]:
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



### 2. Parse the webpage data

In [26]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url).read().decode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

table = soup.body.table.tbody

### 3. Extract data from webpage

In [27]:
def get_cell(element):
    cells = element.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def get_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = get_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

### 4. Create dataframe with the data

In [28]:

data = get_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
d = pd.DataFrame(data, columns=columns)
d.head()
#df.shape

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 5. Data preprocessing

In [29]:
df1 = d[d.Borough != 'Not assigned']
df1 = df1.sort_values(by=['Postcode','Borough'])

df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)

df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [30]:
df_p = df1['Postcode']
df_p.drop_duplicates(inplace=True)
df = pd.DataFrame(df_p)
df['Borough'] = '';
df['Neighbourhood'] = '';


df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df.index:
    for j in df1.index:
        if df.iloc[i, 0] == df1.iloc[j, 0]:
            df.iloc[i, 1] = df1.iloc[j, 1]
            df.iloc[i, 2] = df.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df.index:
    s = df.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df.iloc[i,2 ] = s

### 6. Number of rows

In [31]:
df.shape

(103, 3)

In [32]:
df.to_csv('week3_1.csv', encoding='utf-8', index=False)